In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
import time
from datetime import datetime, timedelta
import numpy as np
from typing import Dict, List, Optional
import json

# Configuration
st.set_page_config(
    page_title="Stock Market Dashboard",
    page_icon="📈",
    layout="wide",
    initial_sidebar_state="expanded"
)

class StockDataFetcher:
    """Handles fetching stock data from various APIs"""
    
    def __init__(self):
        self.base_url = "https://api.polygon.io/v1"
        # Using Alpha Vantage as primary (free tier available)
        self.alpha_vantage_key = "demo"  # Replace with your API key
        self.alpha_vantage_base = "https://www.alphavantage.co/query"
        
    def get_stock_data(self, symbol: str, interval: str = "1min") -> Optional[pd.DataFrame]:
        """Fetch real-time stock data"""
        try:
            # Using Alpha Vantage API (demo data)
            params = {
                'function': 'TIME_SERIES_INTRADAY',
                'symbol': symbol,
                'interval': interval,
                'apikey': self.alpha_vantage_key,
                'outputsize': 'compact'
            }
            
            response = requests.get(self.alpha_vantage_base, params=params)
            data = response.json()
            
            if 'Time Series (1min)' in data:
                time_series = data['Time Series (1min)']
                df = pd.DataFrame.from_dict(time_series, orient='index')
                df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
                df.index = pd.to_datetime(df.index)
                df = df.astype(float)
                df = df.sort_index()
                return df
            else:
                # Fallback to simulated data for demo
                return self.generate_demo_data(symbol)
                
        except Exception as e:
            st.error(f"Error fetching data: {str(e)}")
            return self.generate_demo_data(symbol)
    
    def generate_demo_data(self, symbol: str) -> pd.DataFrame:
        """Generate realistic demo data for testing"""
        np.random.seed(42)  # For reproducible demo data
        
        # Base price for different stocks
        base_prices = {
            'AAPL': 150.0,
            'GOOGL': 2800.0,
            'MSFT': 330.0,
            'TSLA': 800.0,
            'AMZN': 3200.0,
            'NVDA': 450.0,
            'META': 320.0,
            'NFLX': 400.0
        }
        
        base_price = base_prices.get(symbol, 100.0)
        
        # Generate 100 data points for the last 100 minutes
        dates = pd.date_range(
            start=datetime.now() - timedelta(minutes=100),
            end=datetime.now(),
            freq='1min'
        )
        
        # Generate realistic price movement
        returns = np.random.normal(0, 0.002, len(dates))
        prices = [base_price]
        
        for i in range(1, len(dates)):
            price = prices[-1] * (1 + returns[i])
            prices.append(price)
        
        # Create OHLC data
        data = []
        for i, price in enumerate(prices):
            noise = np.random.normal(0, 0.001, 4)
            open_price = price * (1 + noise[0])
            high_price = price * (1 + abs(noise[1]))
            low_price = price * (1 - abs(noise[2]))
            close_price = price * (1 + noise[3])
            
            # Ensure high >= open,close and low <= open,close
            high_price = max(high_price, open_price, close_price)
            low_price = min(low_price, open_price, close_price)
            
            volume = np.random.randint(10000, 100000)
            
            data.append({
                'Open': open_price,
                'High': high_price,
                'Low': low_price,
                'Close': close_price,
                'Volume': volume
            })
        
        df = pd.DataFrame(data, index=dates)
        return df

class TechnicalIndicators:
    """Calculate various technical indicators"""
    
    @staticmethod
    def sma(data: pd.Series, window: int) -> pd.Series:
        """Simple Moving Average"""
        return data.rolling(window=window).mean()
    
    @staticmethod
    def ema(data: pd.Series, window: int) -> pd.Series:
        """Exponential Moving Average"""
        return data.ewm(span=window).mean()
    
    @staticmethod
    def rsi(data: pd.Series, window: int = 14) -> pd.Series:
        """Relative Strength Index"""
        delta = data.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        
        avg_gain = gain.rolling(window=window).mean()
        avg_loss = loss.rolling(window=window).mean()
        
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    @staticmethod
    def macd(data: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9) -> Dict:
        """MACD Indicator"""
        ema_fast = TechnicalIndicators.ema(data, fast)
        ema_slow = TechnicalIndicators.ema(data, slow)
        
        macd_line = ema_fast - ema_slow
        signal_line = TechnicalIndicators.ema(macd_line, signal)
        histogram = macd_line - signal_line
        
        return {
            'macd': macd_line,
            'signal': signal_line,
            'histogram': histogram
        }
    
    @staticmethod
    def bollinger_bands(data: pd.Series, window: int = 20, num_std: float = 2) -> Dict:
        """Bollinger Bands"""
        sma = TechnicalIndicators.sma(data, window)
        std = data.rolling(window=window).std()
        
        upper_band = sma + (std * num_std)
        lower_band = sma - (std * num_std)
        
        return {
            'upper': upper_band,
            'middle': sma,
            'lower': lower_band
        }

class Dashboard:
    """Main dashboard class"""
    
    def __init__(self):
        self.data_fetcher = StockDataFetcher()
        self.indicators = TechnicalIndicators()
        
    def create_candlestick_chart(self, df: pd.DataFrame, symbol: str) -> go.Figure:
        """Create candlestick chart with technical indicators"""
        fig = make_subplots(
            rows=3, cols=1,
            subplot_titles=('Price Chart', 'Volume', 'RSI'),
            vertical_spacing=0.1,
            row_heights=[0.6, 0.2, 0.2]
        )
        
        # Candlestick chart
        fig.add_trace(
            go.Candlestick(
                x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                name=f'{symbol} Price'
            ),
            row=1, col=1
        )
        
        # Moving averages
        sma_20 = self.indicators.sma(df['Close'], 20)
        sma_50 = self.indicators.sma(df['Close'], 50)
        
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=sma_20,
                mode='lines',
                name='SMA 20',
                line=dict(color='orange', width=1)
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=sma_50,
                mode='lines',
                name='SMA 50',
                line=dict(color='blue', width=1)
            ),
            row=1, col=1
        )
        
        # Bollinger Bands
        bb = self.indicators.bollinger_bands(df['Close'])
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=bb['upper'],
                mode='lines',
                name='BB Upper',
                line=dict(color='gray', width=1, dash='dash'),
                showlegend=False
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=bb['lower'],
                mode='lines',
                name='BB Lower',
                line=dict(color='gray', width=1, dash='dash'),
                fill='tonexty',
                fillcolor='rgba(128,128,128,0.1)',
                showlegend=False
            ),
            row=1, col=1
        )
        
        # Volume
        fig.add_trace(
            go.Bar(
                x=df.index,
                y=df['Volume'],
                name='Volume',
                marker_color='lightblue'
            ),
            row=2, col=1
        )
        
        # RSI
        rsi = self.indicators.rsi(df['Close'])
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=rsi,
                mode='lines',
                name='RSI',
                line=dict(color='purple', width=2)
            ),
            row=3, col=1
        )
        
        # RSI reference lines
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)
        
        fig.update_layout(
            title=f'{symbol} Stock Analysis',
            height=800,
            showlegend=True,
            xaxis_rangeslider_visible=False
        )
        
        return fig
    
    def create_metrics_display(self, df: pd.DataFrame) -> Dict:
        """Calculate and return key metrics"""
        current_price = df['Close'].iloc[-1]
        prev_close = df['Close'].iloc[-2]
        change = current_price - prev_close
        change_pct = (change / prev_close) * 100
        
        day_high = df['High'].max()
        day_low = df['Low'].min()
        volume = df['Volume'].sum()
        
        return {
            'current_price': current_price,
            'change': change,
            'change_pct': change_pct,
            'day_high': day_high,
            'day_low': day_low,
            'volume': volume
        }
    
    def run(self):
        """Main dashboard interface"""
        st.title("📈 Real-Time Stock Market Dashboard")
        st.markdown("---")
        
        # Sidebar
        st.sidebar.header("Dashboard Controls")
        
        # Stock selection
        popular_stocks = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN', 'NVDA', 'META', 'NFLX']
        selected_stock = st.sidebar.selectbox(
            "Select Stock Symbol",
            popular_stocks,
            index=0
        )
        
        # Custom stock input
        custom_stock = st.sidebar.text_input("Or enter custom symbol")
        if custom_stock:
            selected_stock = custom_stock.upper()
        
        # Time interval
        interval = st.sidebar.selectbox(
            "Time Interval",
            ['1min', '5min', '15min', '30min', '1hour'],
            index=0
        )
        
        # Auto-refresh
        auto_refresh = st.sidebar.checkbox("Auto Refresh (30s)", value=True)
        
        if st.sidebar.button("🔄 Refresh Data"):
            st.rerun()
        
        # Main content
        col1, col2, col3, col4 = st.columns(4)
        
        # Fetch data
        with st.spinner(f"Fetching data for {selected_stock}..."):
            df = self.data_fetcher.get_stock_data(selected_stock, interval)
        
        if df is not None and not df.empty:
            metrics = self.create_metrics_display(df)
            
            # Display key metrics
            with col1:
                st.metric(
                    "Current Price",
                    f"${metrics['current_price']:.2f}",
                    f"{metrics['change']:+.2f} ({metrics['change_pct']:+.2f}%)"
                )
            
            with col2:
                st.metric("Day High", f"${metrics['day_high']:.2f}")
            
            with col3:
                st.metric("Day Low", f"${metrics['day_low']:.2f}")
            
            with col4:
                st.metric("Volume", f"{metrics['volume']:,}")
            
            # Main chart
            fig = self.create_candlestick_chart(df, selected_stock)
            st.plotly_chart(fig, use_container_width=True)
            
            # Technical indicators section
            st.header("📊 Technical Indicators")
            
            col1, col2 = st.columns(2)
            
            with col1:
                st.subheader("Moving Averages")
                sma_20 = self.indicators.sma(df['Close'], 20).iloc[-1]
                sma_50 = self.indicators.sma(df['Close'], 50).iloc[-1]
                ema_12 = self.indicators.ema(df['Close'], 12).iloc[-1]
                
                st.write(f"**SMA 20:** ${sma_20:.2f}")
                st.write(f"**SMA 50:** ${sma_50:.2f}")
                st.write(f"**EMA 12:** ${ema_12:.2f}")
                
                # Trend analysis
                if sma_20 > sma_50:
                    st.success("📈 Bullish Trend (SMA 20 > SMA 50)")
                else:
                    st.error("📉 Bearish Trend (SMA 20 < SMA 50)")
            
            with col2:
                st.subheader("Momentum Indicators")
                rsi = self.indicators.rsi(df['Close']).iloc[-1]
                
                st.write(f"**RSI (14):** {rsi:.2f}")
                
                if rsi > 70:
                    st.warning("⚠️ Overbought (RSI > 70)")
                elif rsi < 30:
                    st.warning("⚠️ Oversold (RSI < 30)")
                else:
                    st.info("✅ Neutral RSI")
                
                # MACD
                macd_data = self.indicators.macd(df['Close'])
                macd_current = macd_data['macd'].iloc[-1]
                signal_current = macd_data['signal'].iloc[-1]
                
                st.write(f"**MACD:** {macd_current:.3f}")
                st.write(f"**Signal:** {signal_current:.3f}")
                
                if macd_current > signal_current:
                    st.success("📈 Bullish MACD")
                else:
                    st.error("📉 Bearish MACD")
            
            # Raw data section
            with st.expander("📋 Raw Data"):
                st.dataframe(df.tail(20))
            
            # Auto-refresh functionality
            if auto_refresh:
                time.sleep(30)
                st.rerun()
                
        else:
            st.error("❌ Unable to fetch data. Please check the symbol and try again.")
        
        # Footer
        st.markdown("---")
        st.markdown(
            "💡 **Note:** This dashboard uses demo data for demonstration. "
            "For live data, please configure with your preferred stock data API."
        )

# Run the dashboard
if __name__ == "__main__":
    dashboard = Dashboard()
    dashboard.run()